# Pyro事始め
Pyroでは確率モデルを実装することができます。<br>
Pytorchで実装されている初等的な確率関数を利用でき、高速な計算やautogradが利用できます。<br>
また、transformsを用いることで好きな関数を設計することもできます。<br>

In [1]:
import torch
from torch import autograd
import pyro.distributions as dist
import pyro

In [102]:
a = torch.distributions.Bernoulli(0.5).sample()
a.item()

0.0

In [88]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 15.0, 'sunny': 25.0}[cloudy]
    scale_temp = {'cloudy': 5.0, 'sunny': 10.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

In [101]:
weather()

('sunny', 16.511615753173828)

In [16]:
mu = torch.zeros(1)
sigma = torch.ones(1)

In [17]:
x = dist.normal(mu, sigma)

AttributeError: module 'pyro.distributions' has no attribute 'normal'

In [18]:
def model(data):
    # set prior
    alpha0 = torch.tensor(5.0)
    beta0 = torch.tensor(5.0)
    prior = dist.Beta(alpha0, beta0)

    # sample z from the prior
    z = pyro.sample("z", prior)

    with pyro.plate("data_plate", len(data)):
        # observe data
        pyro.sample("obs", dist.Bernoulli(z), obs=data)

In [19]:
d = torch.randn(10)
model(d)